In [1]:
import pandas as pd

# Load datasets for each city
df_city1 = pd.read_csv("C:/Users/acer/Downloads/Delivery Five Cities Datasets/Delivery Five Cities Datasets/delivery_sh.csv")
df_city2 = pd.read_csv("C:/Users/acer/Downloads/Delivery Five Cities Datasets/Delivery Five Cities Datasets/delivery_yt.csv")
df_city3 = pd.read_csv("C:/Users/acer/Downloads/Delivery Five Cities Datasets/Delivery Five Cities Datasets/delivery_hz.csv")
df_city4 = pd.read_csv("C:/Users/acer/Downloads/Delivery Five Cities Datasets/Delivery Five Cities Datasets/delivery_cq.csv")
df_city5 = pd.read_csv("C:/Users/acer/Downloads/Delivery Five Cities Datasets/Delivery Five Cities Datasets/delivery_delivery_jl.csv")

In [2]:
df_city1['city'] = 'City1'
df_city2['city'] = 'City2'
df_city3['city'] = 'City3'
df_city4['city'] = 'City4'
df_city5['city'] = 'City5'

In [ ]:
merged_df = pd.concat([df_city1, df_city2, df_city3, df_city4, df_city5], ignore_index=True)

In [20]:
print(merged_df.info())
print(merged_df['city'].value_counts())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4514661 entries, 0 to 4514660
Data columns (total 17 columns):
 #   Column             Dtype         
---  ------             -----         
 0   order_id           int64         
 1   region_id          int64         
 2   city               object        
 3   courier_id         int64         
 4   lng                float64       
 5   lat                float64       
 6   aoi_id             int64         
 7   aoi_type           int64         
 8   accept_time        datetime64[ns]
 9   accept_gps_time    object        
 10  accept_gps_lng     float64       
 11  accept_gps_lat     float64       
 12  delivery_time      object        
 13  delivery_gps_time  object        
 14  delivery_gps_lng   float64       
 15  delivery_gps_lat   float64       
 16  ds                 int64         
dtypes: datetime64[ns](1), float64(6), int64(6), object(4)
memory usage: 585.6+ MB
None
city
City3    1861600
City1    1483864
City4     931351
Ci

In [ ]:
merged_df.to_csv('merged_city_data.csv', index=False)

In [19]:
merged_df = merged_df.drop_duplicates()

In [15]:
print(merged_df.value_counts())

order_id  region_id  city   courier_id  lng        lat       aoi_id  aoi_type  accept_time     accept_gps_time  accept_gps_lng  accept_gps_lat  delivery_time   delivery_gps_time  delivery_gps_lng  delivery_gps_lat  ds  
0         154        City4  3253        106.26649  29.28891  21674   1         10-23 09:47:00  10-23 09:47:00   106.27495       29.24599        10-23 19:57:00  10-23 19:57:00     106.28845         29.26007          1023    1
3009808   19         City3  4466        120.15066  30.33448  3427    1         08-11 08:16:00  08-11 08:16:00   120.13945       30.32906        08-11 17:25:00  08-11 17:25:00     120.15125         30.33431          811     1
3009814   3          City3  2791        120.10391  30.27025  52881   1         05-13 08:18:00  05-13 08:18:00   120.09711       30.27140        05-13 08:54:00  05-13 08:54:00     120.10410         30.27065          513     1
3009813   105        City4  3689        106.58963  29.74705  1675    14        06-02 15:32:00  06-02 15:3

In [16]:
# Attempt to standardize the format (adjust as needed for your dataset)
merged_df['accept_time'] = pd.to_datetime(merged_df['accept_time'], format='%m-%d %H:%M:%S', errors='coerce')



In [17]:
print(merged_df.dtypes)


order_id                      int64
region_id                     int64
city                         object
courier_id                    int64
lng                         float64
lat                         float64
aoi_id                        int64
aoi_type                      int64
accept_time          datetime64[ns]
accept_gps_time              object
accept_gps_lng              float64
accept_gps_lat              float64
delivery_time                object
delivery_gps_time            object
delivery_gps_lng            float64
delivery_gps_lat            float64
ds                            int64
dtype: object


In [18]:
# Assuming 'city' is the column containing city or region information
# Replace 'city' with the actual column name representing city or region

# Identify columns with missing values
missing_columns = merged_df.columns[merged_df.isnull().any()].tolist()
print("Columns with missing values:", missing_columns)

# Group by city (or region) and compute the mean for each group
city_mean = merged_df.groupby('city')[missing_columns].transform('mean')

# Impute missing values with the corresponding city mean
for column in missing_columns:
    merged_df[column] = merged_df[column].fillna(merged_df.groupby('city')[column].transform('mean'))

# Verify if there are any missing values left
print("Missing values after imputation:")
print(merged_df.isnull().sum())


Columns with missing values: ['accept_gps_lng', 'accept_gps_lat']
Missing values after imputation:
order_id             0
region_id            0
city                 0
courier_id           0
lng                  0
lat                  0
aoi_id               0
aoi_type             0
accept_time          0
accept_gps_time      0
accept_gps_lng       0
accept_gps_lat       0
delivery_time        0
delivery_gps_time    0
delivery_gps_lng     0
delivery_gps_lat     0
ds                   0
dtype: int64


In [ ]:
merged_df['aoi_type'] = merged_df['aoi_type'].astype('category')
print(merged_df['aoi_type'].dtype)

In [ ]:
# Calculate delivery_duration in minutes
merged_df['delivery_duration'] = (pd.to_datetime(merged_df['delivery_time']) - pd.to_datetime(merged_df['accept_time'])).dt.total_seconds() / 60

# Detect outliers using Z-score
from scipy.stats import zscore

# Calculate Z-scores
z_scores = zscore(merged_df['delivery_duration'].dropna())  # Drop NA for Z-score calculation
outliers = (z_scores < -3) | (z_scores > 3)  # Outlier threshold
merged_df = merged_df[~outliers]  # Remove outliers

# Or use IQR
Q1 = merged_df['delivery_duration'].quantile(0.25)
Q3 = merged_df['delivery_duration'].quantile(0.75)
IQR = Q3 - Q1

# Remove outliers using IQR
merged_df = merged_df[
    (merged_df['delivery_duration'] >= Q1 - 1.5 * IQR) & 
    (merged_df['delivery_duration'] <= Q3 + 1.5 * IQR)
]


In [ ]:
merged_df = merged_df[
    (merged_df['lat'].between(-90, 90)) & 
    (merged_df['lng'].between(-180, 180))
]


In [ ]:
merged_df = pd.get_dummies(merged_df, columns=['aoi_type', 'city'], drop_first=True)


In [ ]:
print(merged_df.isnull().sum())

In [ ]:
from sklearn.preprocessing import StandardScaler
# Specify the columns to standardize
numerical_features = ['delivery_duration', 'lng', 'lat']  # Add 'distance' if it exists in your dataset

In [ ]:
# Initialize the scaler
scaler = StandardScaler()

# Fit and transform the numerical features
merged_df[numerical_features] = scaler.fit_transform(merged_df[numerical_features])

In [ ]:
# Check the mean and standard deviation after scaling
print(merged_df[numerical_features].mean())
print(merged_df[numerical_features].std())

In [ ]:
import os

# Define the path where you want to save the CSV file
save_path = "C:/Users/acer/Documents/cleaned_delivery_data_standardized.csv"

# Save the DataFrame as a CSV file
merged_df.to_csv(save_path, index=False)

# Confirm the file has been saved
if os.path.exists(save_path):
    print(f"File successfully saved at {save_path}")
else:
    print("File save failed.")